In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

df=pd.read_excel('dataset_filledsupplier_currency_orderday.xlsx')

In [2]:


missing_conditions = df[
    df['발주'].notnull() &  # 발주 일자는 비어있지 않음
    df['미입고 기간'].isnull() &  # 미입고 기간은 비어있음
    df['창고입고'].isnull() & # 창고 입고도 비어있음
    df['선박입고'].isnull()  # 선박 입고도 비어있음

]

print(f"발주 일자는 있지만 미입고 기간, 창고 입고, 선박 입고도 없는 경우: {len(missing_conditions)}개")
df = df.drop(missing_conditions.index)

print(f"삭제된 행의 개수: {len(missing_conditions)}개")
print(f"남은 데이터프레임의 크기: {df.shape}")
#미입고기간으로 처리.
missing_both = df[df['창고입고'].isnull() & df['미입고 기간'].notnull()]

print(f"창고 입고일은 없고 미입고 기간은 명시되어 있어 미입고 기간으로 분류해야 할 경우 : {len(missing_both)}개")
df = df.dropna(subset=['창고입고'])

발주 일자는 있지만 미입고 기간, 창고 입고, 선박 입고도 없는 경우: 1699개
삭제된 행의 개수: 1699개
남은 데이터프레임의 크기: (22922, 32)
창고 입고일은 없고 미입고 기간은 명시되어 있어 미입고 기간으로 분류해야 할 경우 : 1620개


In [3]:
df['창고입고'].isnull().sum()

0

In [4]:
currency=pd.read_excel('currency_data.xlsx')

In [5]:
df.head()

,청구서번호,No.,Subject,Machinery,Assembly,청구품목,Unnamed: 6,Part No.1,Part No.2,청구량,...,창고입고수량,Control No.,입고창고,창고출고,창고출고수량,출고선박,출고운반선,선박입고,선박입고수량,완료 여부
0,COK-BS-DSP-1901004,1,COK-F-DECK-190104-01,CARGO BOOM VANG BLOCK (STBD 하),BLOCK,"MCKISSICK CONSTRUCTION BLOCKS (WIRE SIZE : 5/8"")",NaN,C15S10BS,2103170,2.0,...,2,19-04-077,BW,2019-05-20,2,COK,ONENESS,2019-06-09,2,완료
1,COK-BS-DSP-1901004,2,COK-F-DECK-190104-01,SPANISH BOOM VANG BLOCK (PORT 상),BLOCK,"MCKISSICK CONSTRUCTION BLOCKS (WIRE SIZE : 5/8"")",NaN,C15D10BS,2103180,1.0,...,1,19-04-064,BW,2019-05-20,1,COK,ONENESS편 탁송,2019-06-09,1,완료
2,COK-BS-DSP-1901004,3,COK-F-DECK-190104-01,PURSE BLOCK,TOW BLOCK,WESTEC 20TON TOW BLOCK,NaN,WESTEC 20TON TOW BLOCK,NaN,1.0,...,1,19-04-077,BW,2019-05-20,1,COK,ONENESS,2019-06-09,1,완료
3,COK-BS-DSP-1905036,1,폰페이 재고정정,MAIN ENGINE,POWER PACK AS,GE POWER PACK FORK - E7(B),NaN,40028340,NaN,8.0,...,8,NaN,PN,2019-08-01,1,DEO,HANARO,2019-08-24,1,완료
4,COK-BS-DSP-1905036,2,폰페이 재고정정,MAIN ENGINE,POWER PACK AS,GE POWER PACK FORK - E7(B),NaN,40028340,NaN,8.0,...,8,NaN,PN,2019-08-29,7,ELS,SL ARCHI,2019-09-30,7,완료


In [6]:
currency.head()

,기간,USD,JPY,EUR
0,2019-01-02,1119.0,1019.47,1281.96
1,2019-01-03,1127.7,1044.87,1270.51
2,2019-01-04,1124.5,1046.36,1284.62
3,2019-01-07,1118.6,1038.34,1283.85
4,2019-01-08,1124.1,1028.85,1283.30


In [7]:
# 1. '창고입고'와 '기간'을 datetime 형식으로 변환
df['창고입고'] = pd.to_datetime(df['창고입고'], errors='coerce')
currency['기간'] = pd.to_datetime(currency['기간'], errors='coerce')



In [8]:
# 1. '창고입고'와 가장 가까운 '기간'을 기준으로 병합
df = pd.merge_asof(df.sort_values('창고입고'), 
                   currency.sort_values('기간'), 
                   left_on='창고입고', 
                   right_on='기간', 
                   direction='backward')

# 3. '견적화폐'에 맞는 환율 값을 매칭
def get_exchange_rate(row):
    if row['견적화폐'] == 'USD':
        return row['USD']
    elif row['견적화폐'] == 'JPY':
        return row['JPY']
    elif row['견적화폐'] == 'EUR':
        return row['EUR']
    else:
        return 1

# 4. 새 칼럼 '환율' 추가하여 환율 값 저장
df['환율'] = df.apply(get_exchange_rate, axis=1)


In [11]:

# 5. 결과 확인
print(df[['창고입고', '견적화폐', '환율']].tail(50))

            창고입고 견적화폐       환율
21252 2022-06-29  JPY   944.26
21253 2022-06-29  USD  1299.00
21254 2022-06-29  USD  1299.00
21255 2022-06-30  KRW     1.00
21256 2022-07-03  USD  1297.30
21257 2022-07-03  USD  1297.30
21258 2022-07-03  USD  1297.30
21259 2022-07-03  USD  1297.30
21260 2022-07-03  USD  1297.30
21261 2022-07-03  USD  1297.30
21262 2022-07-03  USD  1297.30
21263 2022-07-03  USD  1297.30
21264 2022-07-03  USD  1297.30
21265 2022-07-03  USD  1297.30
21266 2022-07-03  USD  1297.30
21267 2022-07-03  USD  1297.30
21268 2022-07-03  USD  1297.30
21269 2022-07-03  USD  1297.30
21270 2022-07-03  USD  1297.30
21271 2022-07-03  USD  1297.30
21272 2022-07-03  USD  1297.30
21273 2022-07-03  USD  1297.30
21274 2022-07-03  USD  1297.30
21275 2022-07-03  USD  1297.30
21276 2022-07-03  USD  1297.30
21277 2022-07-03  USD  1297.30
21278 2022-07-03  USD  1297.30
21279 2022-07-03  USD  1297.30
21280 2022-07-03  USD  1297.30
21281 2022-07-03  USD  1297.30
21282 2022-07-03  USD  1297.30
21283 20

In [12]:
print(df[['환율']].isnull().sum())

환율    127
dtype: int64
